In [7]:
import numpy as np
import math
from scipy import optimize
import numba as nb
import cap_equi as func
import interactive_plots as int_plot

import scipy.stats as stats

# For plots:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
plt.style.use('seaborn-whitegrid')
mpl.style.use('seaborn')
prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# PS2, part 1: Welfare effects of capital injection

The exercise consists of the core setup:
* Two periods, $t=0,1$,
* Two types of agents: Consumers and entrepreneurs. 
* **Consumers:**
    * Linear utility $u_c = c_0^c+c_1^c$.
    * Large endowment of consumption goods in each period: $e$. 
    * Labor endowment of $e_l=1$. Inelastically supplied in a competitive labor market at the price $w_1$.
* **Entrepreneurs:**
    * Linear utility $u_e = c_0^e+c_1^e$.
    * Small endowment $n_0$ in period 0. 
    * Borrow $b_1$, invest in capital $k_1$. 
    * Period 1 they hire labor $l_1$ at wage $w_1$.
    * Entrepreneurs utilize capital and labor in $t=1$ with Cobb-Douglas production function 
    $$ \begin{align} 
    y_1 = k_1^{\alpha}l_1^{1-\alpha} \notag
    \end{align} $$
    * **Entrepreneurs are borrowing constrained:**
    $$\begin{align} 
    b_1\leq \lambda \left(y_1-w_1l_1\right), && \lambda\in (0,1) \notag
    \end{align} $$
* Assume that endowment of consumption goods $(e)$ is *sufficiently* large, for the interest rate to be 1 in equilibrium. (We will show what *sufficiently* means here).

## Q1: Three parts. (i) State the entrepreneur's problem. (ii) Argue that entrepreneurs will choose to maximize profits in period 1, a linear function of $k_1$, and (3) restate entrepreneur's problem as a simple linear problem.

### (i) Entrepreneur problem:

Interpret bullets from **entrepreneurs** above as a problem:

$$
\begin{align}
    \max_{c_0,c_1,k_1,l_1,y_1,b_1} \mbox{ } c_0+c_1,& \notag \\ 
    \text{s.t. }\mbox{ }c_0+k_1 &= n_0+b_1 \notag \\  
                        c_1+b_1 &= y_1-w_1l_1 \tag{Problem 1}\\ 
                            y_1 &= k_1^{\alpha}l_1^{1-\alpha} \notag \\ 
                            b_1\leq &\lambda \left(y_1-w_1l_1\right) \notag
\end{align}$$

States that:
1. Maximize consumption (risk-neutrality).
2. Budget $t=0$: Consumption $(c_0)$ and investment $(k_1)$ equals income from endowment $(n_0)$ and borrowing $(b_1)$.
3. Budget $t=1$: Consumption $(c_1)$ and repayment of debt $(b_1)$ equals income from production $(y_1)$ minues wages $(w_1l_1)$.
4. Production is Cobb-Douglas.
5. Entrepreneur is credit-constrained, with $\lambda$ denoting the degree of future profits that can be used as collateral.

Substitute for $c_0$ and $c_1$ into the utility function, using the budgets for $t=0,1$:

$$\begin{align} 
    c_0+c_1 &= n_0+b_1-k_1+y_1-w_1l_1-b_1 \\ 
            &= n_0+y_1-k_1-w_1l_1 
\end{align}$$

Substitue for $y_1$ using the Cobb-Douglas function (remember you should substitute for $y_1$ in both the object function **and** the constraints). The problem is then reduced to:

$$\begin{align}
    \max_{k_1,l_1,b_1} n_0+k_1^{\alpha}l_1^{1-\alpha}-k_1-w_1l_1, && s.t. b_1\leq \lambda \left(k_1^{\alpha}l_1^{1-\alpha}-w_1l_1\right). \tag{Problem 2}
\end{align}$$ 

The Lagrangian function corresponding to this problem is then given by:

$$\begin{align} 
    \mathcal{L}_2 = n_0+k_1^{\alpha}l_1^{1-\alpha}-k_1-w_1l_1+\mu\left[\lambda\left(k_1^{\alpha}l_1^{1-\alpha}-w_1l_1\right)-b_1\right],
\end{align}$$

where $\mu\geq0$ is the shadow-variable related to the credit-constraint. As in problem set, exercise 3.5 in (JT), this problem is characterized by a set of first order conditions with Karush-Kuhn-Tucker conditions for $\mu$ and the credit constraint. This is because of the inequality constraint (instead of standard Lagrangian approach when the constraint is with "=").


### (ii) Optimal choice of labor:

Note that profits in $t=1$ is given by:

$$\begin{align}
    \pi_1 = k_1^{\alpha}l_1^{1-\alpha}-w_1l_1.
\end{align}$$

Optimal choice of labor $(l^*)$ is then characterized by:

$$\begin{align}
    \dfrac{\partial \mathcal{L}}{\partial l_1} = (1-\alpha)k_1^{\alpha}l_1^{-\alpha} -w_1 + \mu\lambda\left[(1-\alpha)k_1^{\alpha}l_1^{-\alpha}-w_1\right] = 0
\end{align}$$

We can rewrite this condition as:

$$\begin{align}
    \left[(1-\alpha)k_1^{\alpha}l_1^{-\alpha}-w_1\right](1+\mu\lambda)=0 && \Rightarrow && w_1 = (1-\alpha)k_1^{\alpha}l_1^{-\alpha}
\end{align}$$

Note that this corresponds to maximizing $\pi_1$ wrt. labor. Note further that we can use this to write up two other terms:

$$\begin{align} 
w_1l_1^* = (1-\alpha)k_1^{\alpha}(l_1^*)^{1-\alpha}, \tag{Labor Share}
\end{align}$$ 

and by isolating for $l_1$:

$$\begin{align}
    l_1^* = \dfrac{(1-\alpha)^{1/\alpha}k_1}{w_1^{1/\alpha}}. \tag{$l^*$}
\end{align}$$

To show that the profit function is linear in $k_1$, when labor is chosen optimally, start by using (Labor Share):

$$\begin{align}
    \pi_1(l=l^*) &= k_1^{\alpha}(l_1^*)^{1-\alpha}-w_1l_1^* \notag \\
                 &= \alpha k_1^{\alpha}(l_1^*)^{1-\alpha}
\end{align}$$

Now substitute for $(l^*)$:

$$\begin{align}
    \pi_1(l=l^*) &= \alpha k_1^{\alpha}\dfrac{(1-\alpha)^{(1-\alpha)/\alpha}k_1^{1-\alpha}}{w_1^{(1-\alpha)/\alpha}} \notag \\ 
                 &= \underbrace{\dfrac{\alpha (1-\alpha)^{(1-\alpha)/\alpha}}{w_1^{(1-\alpha)/\alpha}}}_{\equiv R(w_1)}k_1 \tag{Profit}
\end{align}$$







### (iii) Restate as linear problem:

Using (Profit) in (Problem 2):

$$\begin{align}
    \max_{k_1,b_1}\mbox{ }n_0-k_1+R(w_1)k_1, && \text{s.t. } b_1\leq \lambda R(w_1)k_1 \tag{Problem 3}.
\end{align} $$

## Q2: Derive entrepreneur's demand for capital, and argue how the solution depends on $\lambda$ and $R(w_1)$.

Problem $3$ is linear in $k_1$. Thus capital demand depends on:

* If $R(w_1)<1$: Investment in capital is not profitable, $k_1^*=0$. (it is implicitly assumed that $k\geq0$)
* If $\lambda R(w_1)>1$: Investment in capital is so profitable that borrowing constraint never binds, $k_1^*=\infty$.
* If $\lambda R(w_1)<1<R(w_1)$: Capital is postponed as much as possible, but borrowing constraint will bind. Solution is then to set $c_0=0$ in the budget at $t=0$, and use that $b_1$ is determined by the borrowing constraint:

$$\begin{align} 
    k_1^* &= n_0 +b_1 \\
          &= n_0+\lambda R(w_1)k_1^* \\ 
\Rightarrow k_1^* &= \dfrac{n_0}{1-\lambda R(w_1)}
\end{align}$$

* Finally, if $R(w_1)=1$ the entrepreneur is indifferent between consumption and investment. In this case capital demand is indeterminate:
$$\begin{align}
    k_1^*\in\left[0,\mbox{ }\dfrac{n_0}{1-\lambda}\right].
\end{align}$$

**Illustrating demand for capital, and equilibrium capital**:

Before we proceed to the next questions, let us briefly illustrate how capital demand looks like, and how it behaves in equilibrium. *(NB: You don't need to change anything in the cell with code below (you can though), just use the sliders in the graph.)* 

The first part here plots the capital demand and the interest rate function $(R=\alpha k^{\alpha-1})$. 

In [5]:
# Baseline parameters:
n_0base, lambda_base, alpha_base, Inf = 0.1, 0.55, 0.9, 10**3

# Exogenous grid of R (interest rate):
R_min, R_max, N_R= 0.1, 2, 100
R_grid = np.linspace(R_min, R_max, N_R)

# Upper and lower limit of sliders to operate below:
n_0_lower, n_0_upper, n_0_step = 0, 0.5, 0.05
lambda_lower, lambda_upper, lambda_step = 0, 0.9, 0.05

# Plot:
int_plot.interactive_cap(n_0base, lambda_base, alpha_base, Inf, N_R, R_grid, n_0_lower, n_0_upper, n_0_step, lambda_lower, lambda_upper, lambda_step)

interactive(children=(FloatSlider(value=0.55, description='$\\lambda$', max=0.9, step=0.05), FloatSlider(value…

The second part here plots the equilibrium value across $(\lambda,n)$. 

In [6]:
# Grids:
n_0_lower, n_0_upper, N_n = 0, 0.5, 51  
lambda_lower, lambda_upper, N_lambda = 0, 0.9, 91
n_grid = np.linspace(n_0_lower, n_0_upper,N_n, dtype=np.float)
lambda_grid, lambda_step = np.linspace(lambda_lower, lambda_upper, N_lambda, retstep=True, dtype=np.float)
# baseline:
n_0base, lambda_base, alpha = 0.1, 0.5, 0.9
# Plot:
int_plot.interactive_equicap(N_n,N_lambda,alpha,n_grid,lambda_grid,math.ceil(N_lambda/2),lambda_lower, lambda_upper,lambda_step)

interactive(children=(FloatSlider(value=0.46, description='$\\lambda$', max=0.9, step=0.01), Output()), _dom_c…

## Q3: Show the cutoff $n^*$, where $n_0 \geq n^*$$ implies that the first-best level of capital can be financed, and the borrowing constraint is not binding.

Let us define the first-best level of capital. Recall that in equilibrium $l_1=1$ (as this is the endowment supplied by consumers). Define first-best as **the level of capital that maximizes net output**:

$$\begin{align} 
    \max_{k_1} k_1^{\alpha}-k_1 && \Rightarrow && k_1^{fb} = \alpha^{1/(1-\alpha)}.
\end{align}$$

In equilibrium the interest rate $R(w_1)$ is defined by the marginal product of capital:

$$\begin{align}
    R(w_1) = \alpha k_1^{\alpha-1}, && \Rightarrow && k_1 = \left(\dfrac{\alpha}{R(w_1)}\right)^{1/(1-\alpha)}
\end{align}$$

Thus for $k_1=k_1^{fb}$ we need $R(w_1)=1$. When $R(w_1)=1$ the capital demand was indeterminate, i.e. it covered a range of possible solutions. The optimal level $k_1^{fb}$ is one of the feasible capital demand levels as long as:

$$\begin{align}
k_1^{fb}\equiv \alpha^{1/(1-\alpha)}\in\left[0,\mbox{ }\dfrac{n_0}{1-\lambda}\right]. 
\end{align}$$

As long as $n_0$ is sufficiently large the first best is thus an equilibrium. The threshold $n^*$ where the first best is an equilibrium for $n_0\geq n^*$ is then given by the condition above with equality:

$$\begin{align}
    \alpha^{1/(1-\alpha)} = \dfrac{n^*}{1-\lambda} && \Rightarrow && n^* = (1-\lambda)\alpha^{1/(1-\alpha)} 
\end{align}$$



**Addtional remark:** Give another explanation for why we need R=1 for optimality:

Optimality requires that the $\color{blue}{\text{marginal rate of transformation}}$ equals $\color{blue}{\text{the marginal rate of substitution}}$

## Q4: Show that if n<n* there is an equilibrium where the entrepreneurs are constrained and $k_1$ becomes an increasing function of $n_0$.

* **Capital demand:** Recall that when $n_0<n^*$ the entrepreneurs do not have sufficient equity; thus they are credit constrained.
* In the credit constrained case recall that capital was given by the budget constraint for $t=0$ with $c_0=0$ imposed, i.e.:

$$\begin{align}
    k_1 = n_0+\lambda R(w_1) k_1.
\end{align}$$ 

Substituting for $R(w_1)$ this yields:

$$\begin{align}
    k_1 - \lambda\alpha k_1^{\alpha}=n_0. \tag{$k_1^*$, credit-constrained}
\end{align}$$

As long as $n_0<n^*$ this equations determines $k_1$. Differentiation wrt. $n_0$ then yields

$$\begin{align}
    \dfrac{\partial k_1}{\partial n_0}-\lambda \alpha^2k_1^{\alpha-1}\dfrac{\partial k_1}{\partial n_0}=1 && \Rightarrow && \dfrac{\partial k_1}{\partial n_0} = \left(1-\lambda \alpha k_1^{\alpha}\dfrac{\alpha}{k_1}\right)^{-1}.
\end{align}$$

Using the expression above for the credit constrained $k_1$ we can rearrange slightly to show that

$$\begin{align}
\dfrac{\partial k_1}{\partial n_0} = \dfrac{1}{1-\alpha(k_1-n_0)/k_1}>1
\end{align}$$

Capital thus increases more than 1-to-1 with the entrepreneur's equity here. The reason for this is the following:

In the credit constrained case the entrepreneur prefers to allocate **all** of his consumption to $t=1$: Thus when $n_0$ increases marginally so does $k_1$, initially 1-to-1. When $k_1$ increases the entrepreneur increases his future profits $(y_1-w_1l_1)$; this relaxes the credit-constraint, allowing him to borrow more against future profits. This increases investments further.  

## Q5: Introduce a lump-sum tax on consumers in period 0 that transfers wealth to entrepreneurs. Derive expected utility as a function of the lump sum tax rate.

Let us start with the consumers' utility. This is straightforward:

$$\begin{align}
    U_C(\tau) = 2e-\tau + \overbrace{(1-\alpha)k_1^{\alpha}}^{\equiv w_1(\tau)}
\end{align}$$

**The entrepreneur part:** 

Setup is as defined in (Problem 3) earlier, simply with an added $\tau$:

$$\begin{align}
    U_E(\tau) = n_0+\tau +k_1\left[R(w_1)-1\right] 
\end{align}$$

Substitute for the equilibrium interest rate we then have

$$\begin{align}
    U_E(\tau) = n_0+\tau + \alpha k_1^{\alpha}-k_1
\end{align}$$

Now we have two cases:

### The credit-constrained case

If $n_0+\tau<n^*$: The entrepreneur is **credit-constrained**, and capital is defined as earlier by the equality 

$$\begin{align}
k_1-\lambda \alpha k_1^{\alpha} = n_0+\tau.
\end{align}$$

Substituting for this we get

$$\begin{align}
    U_E(\tau) = \alpha k_1^{\alpha}(1-\lambda)
\end{align}$$


Increasing the tax $\tau$ corresponds to increasing the available endowment for entrepreneurs at $t=0$. As we confirmed above, capital is increasing in $n_0$ and thus in $\tau$, when the credit constraint is binding. Thus $\partial U_E/\partial\tau>0$ when the entrepreneur is constrained. Technically we can confirm this:


$$\begin{align}
    \dfrac{\partial U_E}{\partial \tau} &= \alpha^2k_1^{\alpha-1}(1-\lambda)\dfrac{\partial k_1}{\partial n_0} \\
                                        &= \alpha R(w_1)(1-\lambda)\dfrac{\partial k_1}{\partial n_0}>0
\end{align}$$

Using the derivative $\partial k_1/\partial n_0$, we can (if we prefer) simplify this further as

$$\begin{align}
    \dfrac{\partial U_E}{\partial \tau} &= \dfrac{\alpha R(w_1)(1-\lambda)}{1-\alpha \lambda R(w_1)}>0, && n_0+\tau<n^*
\end{align}$$



### The non-constrained case: 

If $n_0+\tau\geq n^*$ the capital equilibrium is unaffected. In this case the tax simply transfers wealth from consumers to entrepreneurs, who then immediately consume it. In other words we sum up:

$$\begin{align}
    \dfrac{\partial U_e}{\partial \tau} = \left\lbrace \begin{array}{ll}  1 & n_0+\tau\geq n^* \\
    \dfrac{\alpha R(w_1)(1-\lambda)}{1-\alpha \lambda R(w_1)} & n_0+\tau<n^* \end{array}\right.
\end{align}$$

## Q6: Show that the relationship between consumer utility and the lump-sum tax is non-monotone. Show that for sufficiently small $n_0$ the tax can be pareto-improving.

Entrepreneurs always benefit from the tax. For consumers the tax includes three potential effects:

$$\begin{align}
    \dfrac{\partial U_c}{\partial \tau} = \left\lbrace \begin{array}{ll} \color{red}{-1}, & n_0+\tau\geq n^* \\ \color{red}{-1} +\color{blue}{(1-\alpha)R(w_1)}\color{blueviolet}{\dfrac{\partial k_1}{\partial \tau}}, & n_0+\tau<n^*\end{array}\right.
\end{align}$$

When $n_0+\tau\geq n^*$ the effect is clearly negative. However, when entrepreneurs are credit-constrained:

1. Negative wealth transfer effect: $\color{red}{-1}$.
2. Entrepreneurs increase capital investment, which increases labor wages: $\color{blue}{(1-\alpha)R(w_1)}$.
3. **(Credit multiplier)** The transfer loosens credit constraitn incuding further investment: $\color{blueviolet}{\partial k_1/\partial \tau}>1$.

If effects 2-3. dominate effect 1., the tax has a pareto-improving effect. Effects 2-3. are largest for small values of $k_1$ (thus $n_0$).



## Q7: Discuss how this justifies government's capital injection during the financial crisis. What features might the model be missing?

The role for capital injections in this model:

* A capital injection relaxes the banks' borrowing constraint, increases lending to the real sector and thus investments. This increases employment (not in our model though) and wages, which can offset the effects of the recession. 

* *Caveats (a few of potentially many):*
    1. Moral hazard (reputation effect): If financial institutions expect bailouts $\Rightarrow$ increase risky investments.
    2. Endogenous uncertainty: A feature of the financial crisis was precautionary behavior in light of increased uncertainty. The capital injection reduces uncertainty of asset prices.

## Q8: How does key findings in Q6 depend on the type of borrowing constraint?

The short answer: The borrowing constraint is quite standard. There are microfoundations that lead to this type of constraint.

In this model future profits can (partly) be collateralized. There are several potential explanations why future profits cannot entirely be collateralized.

* **Informational/enforcement frictions** are at the heart of any borrowing constraint (to the best of my knowledge at least).
* **Moral Hazard**: Behave/shirk.
* **Collateralized assets** as Kiyotaki and Moore (1997) leads to similar results. 

# PS2, part 2: Costly state verification and optimality of debt

The core of the model setup:

* **Utility** is linear: $U_E=c_0+c_1$.
* Entrepreneur invests at time $0$, cost $I$. Return at time $t=1$ is random, distributed according to $g(s)$:

$$\begin{align}
    Return = Rs, && s\sim g(s), && \text{Support on }S\equiv \left[s^{min}, s^{max}\right]
\end{align}$$
Also assumed that
$$\begin{align}
    \mathbb{E}[s]=1, && R>I.
\end{align}$$

* **Asymmetric information:** State $s$ is unknown at time $t=0$. At time $t=1$ the entrepreneur (E) observes the true value of $s$ costlessly. The financier (F) can choose to audit and observe it, but at the cost $\kappa>0$.
* **Assumption:** The financier chooses a *deterministic* audit rule. I.e. no *random/stochastic* audit rule. By the *revelation principle:* Only focus on contracts in which E has no incentive to lie. 
* **Contracts:** Specified by the triplet

$$\begin{align}
    Contract = \left( R^E(s), \mbox{ }R^F(s), \mbox{ }S_A\right), && S_A\subset S
\end{align}$$
where $R^E(s)$ and $R^F(s)$ denotes the payoffs for E,F (net of audit cost). $S_A$ is the set of states $S$, where $F$ chooses to audit.

* **Resource constraint:**

$$\begin{align}
    R^E(s)+R^F(s)+\kappa \mathbf{1}_{s\in S_A} = Rs, && R^E(s)\geq 0.
\end{align}$$

where $\mathbf{1}_{s\in S_A}$ indicates auditing, and the inequality captures *limited liability* of the entrepreneur.

* **Participation constraint:** (for F)

$$\begin{align}
    \int_S R^F(s)g(s)\mbox{ }ds\geq I.
\end{align}$$

States that expected revenue covers investment cost.

### Q1: Write down the IC constraint for E. What does it imply for F's net payoff?

**IC constraint** with a continuous set of possible outcomes: *In any given state $s$ it must not be profitable to deviate to *any* state $s'$.*

Another useful definition is **the revelation principle**. The principle roughly states that every potential equilibrium outcome, can be achieved by an incentive-compatible strategy.

The IC constraint then states that

$$\begin{align}
    Rs-R^F(s)-\kappa \mathbf{1}_{s\in S_A}\geq Rs-R^F(s'), && \forall s'\in\left(S\backslash S_A\right)
\end{align}$$

The left-hand side (LHS) is the profit from E telling the true $s$. The RHS is the outcome if E is in state $s$, but signal to F that he is in state $s'$. Note that we only consider deviations to $s'\in (S\backslash S_A)$. There is no incentive to lie about a state $s'$, where E knows he will be audited. 

**What does it imply for F's net payoff?**

* Consider two states, $s,s'$, both in the no-audit region. The IC constraint for state $s$ states $R^F(s)\geq R^F(s')$. The IC constraint for state $s$ states $R^F(s')\geq R^F(s)$. 

* For them both to hold simultaneously, there must be exactly one payoff level for all $s\notin S_A$. Denote this $R^F(\bar{s})$. If there was a state with higher payoffs in the region $S\backslash S_A$, where E is not audited, he would always report this.


* Now consider the case where $s\in S_A$ (recall that alternative $s'\notin S_A$): The $(IC)$ then states that

$$\begin{align}
    Rs-R^F(s)-\kappa \geq Rs-R^F(\bar{s}) && \Rightarrow && R^F(s)\leq R^F(\bar{s}), && \forall s\in S_A.
\end{align}$$

* This states that in the audit region, the financier (F) has to accept lower returns $R^F(s)$ than in the non-audit region. If this was not the case, E would always report a state in the no-audit region, and thus get higher profits. 

### Q2: Define a standard debt contract:

$$\begin{align}
    R^F(s) = \left\lbrace \begin{array}{ll} R\bar{s}, & s\geq \bar{s} \\ Rs-\kappa, & s\leq \bar{s} \end{array}\right., && S_A=\left[s^{min},\mbox{ }\bar{s}\right].
\end{align}$$

### Check that any level $\bar{s}$ satisfies both the resource constraint and the (IC).

* **(IC) constraint:** 
    * When $s\geq \bar{s}:$ The entrepreneur receives $R(s-\bar{s})$ by telling the truth. The same if he over-reports. If he under-reports in the audit-region he receives 0.
    * When $s<\bar{s}:$ If E over-reports (in the no-audit region) he receives $R(s-\bar{s})<0$. By true reporting he receives 0. 

    
* **Resource constraint:** 
    * If $s\geq \bar{s}$ the resource constraint reads:
    $$\begin{align}
        \overbrace{R(s-\bar{s})}^{R^E}+\overbrace{R\bar{s}}^{R^F} + \overbrace{0}^{audit}= Rs
    \end{align}$$
    * If $s<\bar{s}$ the resource constraint reads:
    
$$\begin{align}
    \overbrace{0}^{R^E} +\overbrace{Rs-\kappa}^{R^F} + \overbrace{\kappa}^{audit} = Rs
\end{align}$$






### Q3: Prove that the standard contract is optimal.

Consider a non-standard contract:

$$\begin{align} 
    (\tilde{R}^E, \tilde{R}^F, \tilde{S}_A)
\end{align}$$

* The (IC) constraint still needs to hold. Thus for $s\notin \tilde{S}_A$ the non-standard contract still has to offer the same return. Denote this constant $\tilde{R}^F$.
* Note furthermore that $\tilde{S}_A$ has to be a closed interval on the form $\tilde{S}_A = [s^{min},\tilde{s}]$: Assume there is a level $s$ that is not audited, but $s'>s$ is audited. If E is in state s', he can report $s$ without being audited and profit from it.
* This implies that $\tilde{R}^F=R\tilde{s}$. 

Consider now the various regions of $s$:

* For $s\in\left(\tilde{S}_A \cap S_A\right):$ F receives $Rs-\kappa$ in both cases.
* For $s\in\left(\tilde{S}_A \cap (S\backslash S_A)\right):$ F receives higher payoff in standard debt contract, where he does not pay the audit cost.
* For $s$ in non-audit region: Let the standard contract offer the level $\bar{R}^F= \tilde{R}^F$. Thus F is indifferent in these cases.

For entrepreneurs: The standard debt contract minimizes audit-costs (given IC and participation constraints). Given that they receive the residual income F's payment, E prefers a standard contract as well. 



### Q4: Assume that $s$ is uniform. Characterizes the riskiness of the optimal debt contract.

Assume that $s$ is uniform on $[1-\Delta, 1+\Delta]$. In other words the pdf (f) and cdf (F) are given by:

$$\begin{align}
f(s) = \dfrac{1}{2\Delta}, && F(s) = \dfrac{s-(1-\Delta)}{2\Delta}, && s\in [1-\Delta, 1+\Delta]
\end{align}$$

The entrepreneur then solves:

$$\begin{align}
    \max_{\bar{s}} \dfrac{1}{2\Delta} \int_{\bar{s}}^{1+\Delta}(Rs-R\bar{s})\mbox{ }ds
\end{align}$$

subject to F's participation constraint

$$\begin{align}
    \dfrac{1}{2\Delta} \int_{\bar{s}}^{1+\Delta} R\bar{s}\mbox{ }ds+\dfrac{1}{2\Delta}\int_{1-\Delta}^{\bar{s}}(Rs-\kappa)\mbox{ }ds\geq I
\end{align}$$

Integrating out the expression we write this as:

$$\begin{align}
       \left[1+\Delta-\bar{s}\right]R\bar{s}+\left[\dfrac{R}{2}\bar{s}^2-\bar{s}\kappa\right]-\left[\dfrac{R}{2}(1-\Delta)^2-(1-\Delta)\kappa\right]\geq 2\Delta I,
\end{align}$$

or rewritten as

$$\begin{align}
    -\dfrac{R}{2}\bar{s}^2+\left[R(1+\Delta)-\kappa\right]\bar{s}+(1-\Delta)\kappa-\dfrac{R}{2}(1-\Delta)^2\geq 2\Delta I. \tag{PC}
\end{align}$$



We note here that:
* For $s<\bar{s}:$ the entrepreneur receives 0. Thus the object function integrates from $\bar{s}$ to $1+\Delta$.
* For $\bar{s}\leq s \leq 1+\Delta$, the financier receives $R\bar{s}$. In the non-audit region he receives $Rs-\kappa$.

It is straightforward to see that for the entrepreneur, the lowest possible $\bar{s}$ is preferred, where the participation constraint still holds.

To do this proceed as follows:

* If $I$ is sufficiently small, the obvious solution is to choose $\bar{s}=1-\Delta$. This is feasible (participation constraint holds) if:

$$\begin{align}
        R(1-\Delta)\geq I
\end{align}$$

* There are two options now: Either there is **no** solution within $[1-\Delta, 1+\Delta]$. Otherwise the solution is given in the case where $(PC)$ holds with equality.

Solving for $(PC)$ with equality, we have:

$$\begin{align}
 \bar{s}^* = 1+\Delta-\dfrac{\kappa}{R}-\dfrac{1}{R}\sqrt{(R(1+\Delta)-\kappa)^2+2R*\left[(1-\Delta)\kappa-\dfrac{R}{2}(1-\Delta)^2-2\Delta I\right]}, \tag{$s^*$}
\end{align}$$

Note that there might not be any solution at all, if the term in the $\sqrt{\cdot}$ is negative. 



In [4]:
# Parameter values:
R = 2
I = 1.5
Delta = 0.5
kappa = 0.1
# Grid of sbar:
sbar = np.linspace(1-Delta, 1+Delta, 101)
# PC constraint:
int_plot.interactive_pc(R,I,Delta,kappa,sbar)
s_star = 1+Delta-kappa/R-(1/R)*math.sqrt((R*(1+Delta)-kappa)**2+2*R*((1-Delta)*kappa-(R/2)*(1-Delta)**2-2*Delta*I))
print(s_star)

interactive(children=(FloatSlider(value=1.5, description='I', max=2.0), FloatSlider(value=0.1, description='$\…

0.8155711229775239


In the figure above (with values $R=2,I=1.5,\Delta=0.5,\kappa=0.1$) note that the PC constraint holds for values of $\bar{s}$ slightly larger 0.8.

### Q5: Find the threshold cost $I$, where the project is profitable.

Using the result from Q4 the threshold $I^{max}$, where the project is financed as long as $I\leq I^{\max}$ is defined as follows:

* If $R(1-\Delta)\geq I$ then $s^*=1-\Delta$. In this case the largest cost $I$ that is financed, is given by $R(1-\Delta)$.
* If $R(1-\Delta)<I$ then there exists a solution, recall that there exists a solution as long as the term $\sqrt{\cdot}$ in $s^*$ is non-negative.
* Note furthermore, that $I$ enters negatively in that expression; thus the higher $I$, the closer we get to a scenario, where there is no $\bar{s}^*$. 
* In other words, the largest $I$ that allows a feasible solution is defined by the condition:

$$\begin{align}
(R(1+\Delta)-\kappa)^2+2R*\left[(1-\Delta)\kappa-\dfrac{R}{2}(1-\Delta)^2-2\Delta I\right] = 0 && \Rightarrow && I = R-\kappa+\dfrac{\kappa^2}{4\Delta R}.
\end{align}$$

Combining the two potential scenarios the highest possible $I$ that can be financed is thus given by 

$$\begin{align}
     I^{max} = \max \Bigg\lbrace \underbrace{R-\kappa+\dfrac{\kappa^2}{4\Delta R}}_{\text{Scenario 1}},\mbox{ }\mbox{ }\underbrace{R(1-\Delta)}_{\text{Scenario 2}}\Bigg\rbrace.
\end{align}$$

A few comments on this solution:

* **When $\kappa<2R\Delta$:** The max-operator returns *scenario 1*. In this case $I^{max}$ is decreasing in both $\kappa$ and $\Delta$. Why?

    * When $\kappa$ increases the cost of auditing goes up. When $\kappa<2\Delta$ the $(PC)$ constraint is binding implying that $s^*>1-\Delta$; in other words, the optimal debt contract involves some costly auditing. In this scenario a larger audit-cost makes investment projects less profitable over all $\Rightarrow I^{max}$ decreases.
    
    * An increase in $\Delta$ increases the uncertainty of the outcome. You may verify (either from just adjusting the graph above, or solving for the derivative) that $s^*$ in this case is increasing in $\Delta$. In other words, the increase in $\Delta$ increases the region of $s$, where the financier has to audit, in order to ensure zero expected profits. This lowers the profitability of the project for F, which lowers $I^{max}$.
    
* **When $\kappa\geq 2R\Delta$:** The max operator returns *scenario 2*. 

    * In this case $I^{max}$ is simply given by $R(1-\Delta)$: When there is no auditing at all, the project is still profitable for F. Thus any change in auditing-costs $(\kappa)$ does not affect $I^{max}$. 
    
    * An increase in $\Delta$ clearly lowers $s^*$ proportionally. As the financier in this case always receive the simple payoff $R^F=Rs^* = R(1-\Delta)$, an increase in $\Delta$ lowers the profits from the project, thus $I$ decreases. 
   
